# <<<<<<<<<<<<<<<<< MODELOS >>>>>>>>>>>>>>>>>>>>>>>

GENERAR ARCHIVOS DE TRAIN Y TEST

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/processed.csv')
train, test = train_test_split(df, test_size=0.33, shuffle=True) # Pronto llegará la 33

train.to_csv('../data/train.csv', index=False)
test.to_csv('../data/test.csv', index=False)

# 1. LOGISTIC REGRESSION (me falta el cross validation)

Hands on Machine Learning: "Just like a Linear Regression model, a Logistic Regression model computes a weighted sum of the input features (plus a bias term), but instead of outputting the result directly like the Linear Regression model does, it outputs the logistic of this result"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

TRAIN

In [ ]:
train = pd.read_csv('../data/train.csv')
train.head(1)

PREDICTORAS Y TARGET

In [ ]:
X = train.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = train['Visualizaciones']

PREPARACIÓN DE DATOS

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33, shuffle=True)

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test) #predicciones

TEST

In [ ]:
test = pd.read_csv('../data/test.csv')

In [ ]:
X = test.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = test['Visualizaciones']

In [ ]:
predicciones_finales = lr.predict(X)

In [ ]:
#para hacer el dataframe final
DTpredicciones = pd.DataFrame(predicciones_finales)
DTpredicciones.reset_index(drop=True, inplace=True) #para que se quiten los indices
id = test.loc[:, ['Título del vídeo']]
id.reset_index(drop=True, inplace=True)
df_unido = id.join(DTpredicciones) #unir

In [ ]:
df_unido.columns = ['Título del vídeo', 'Visualizaciones'] #para poner el nombre a las columnas
df_unido['Visualizaciones'] = df_unido['Visualizaciones'].round(0).astype('int') #para redondear las visualizaciones y poner la columna en números enteros

In [ ]:
df_unido.head()

In [ ]:
#df_unido.to_csv('siralatriste_submission.csv', index=False) #para pasar el dataframe a csv

PARA GUARDAR EL MODELO

In [ ]:
import pickle
with open('../models/modelo_LR.pkl', 'wb') as archivo:
    pickle.dump(lr, archivo)

# 2. RANDOM FOREST REGRESSOR con GradientBoostingRegressor

Hands on Machine Learning : "Random Forests work by training many Decision Trees on random subsets of the features, then averaging out their predictions. Building a model on top of many other models is called Ensemble Learning, and it is often a great way to push ML algorithms even further"

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv('../data/processed.csv')

""" train, test = train_test_split(df, test_size=0.33, shuffle=True) #con shuffle, que los videos estaban ordenados por visualizaciones """

In [ ]:
""" train = pd.read_csv('data/train.csv')
train.head(1) """

In [ ]:
X = df.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = df['Visualizaciones']

In [ ]:
train.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33, shuffle=True)

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=33)
model.fit(X_train, y_train)

In [ ]:
RandomForestRegressor(random_state=33)

Esto es sin cross validation

In [ ]:
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mae

Con cross validation:

In [ ]:
model_pipeline = Pipeline(steps=[('model', RandomForestRegressor(n_estimators=100, random_state=33))])

In [ ]:
#EVALUATION que hay que pasar al evaluation.py
scores = -1 * cross_val_score(model_pipeline, X, y, cv=5, scoring='neg_mean_absolute_error') 
#lo multiplico por -1 para que se vea positivo 

In [ ]:
scores

In [ ]:
scores.mean()

Guardar el modelo

In [ ]:
import pickle
with open('../models/modelo_RFR.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

# 3. DECISION TREES (DecisionTreeRegressor junto a XGBRegressor, el cual igual quito para ponerlo por separado con el ejemplo de apuntes del test)

Hands On Machine Learning: "This is a powerful model, capable of finding complex nonlinear relationships in the data"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn import metrics

In [ ]:
#OJO QUE AQUI EMPIEZO CON TODOS LOS DATOS (data =) NO CON train =
df = pd.read_csv('../data/processed.csv')
df['Visualizaciones'] = df.Visualizaciones
#data.head()

GRAFICO INTERESANTE PARA VER DE OTRA MANERA LA CORRELACION DE LAS VARIABLES (Hay que recortar las variables a las que se eligieron)

In [ ]:
""" # GRÁFICO PARA VER LA CORRELACION DE LAS VARIABLES
plt.figure(figsize=(5,5))
bars = df.corr()['Visualizaciones'].sort_values(ascending=False).plot(kind='bar') """

SLIPT DEL TEST Y DEL DATA

In [ ]:
X = df.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = df['Visualizaciones']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33, shuffle=True)

In [ ]:
#X_train.head()

FIT DEL MODELO (base)

In [ ]:
regressor = XGBRegressor()
model = regressor.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
test = pd.DataFrame({'Predicted views':y_pred, 'Actual views':y_test})

fig= plt.figure(figsize=(16,8))

test = test.reset_index() # REVISAR ESTE PASO Y EL DE DESPUÉS
test = test.drop(['index'],axis=1)

plt.plot(test[:50]) #revisar si tengo que coger sobre 50 o más bien la mitad de los datos
plt.legend(['Actual views','Predicted views'])

EVALUATION (pasar al .py después, creo)

In [ ]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

DECISION TREE

In [ ]:
param_grid = {
    'max_depth': [None, 2, 3, 4]
}

model = GridSearchCV(
    DecisionTreeRegressor(random_state=33),
    cv=10,
    scoring='neg_mean_squared_error',
    param_grid=param_grid
)

AÑADIR UNA NUEVA FEATURE USANDO EL DECISION TREE COMBINANDO LAS 3 VARIABLES MÁS CORRELADAS

In [ ]:
corr = df.corr().abs() #para sacarlo con valores absolutos

In [ ]:
corr_visual = corr.loc[:,['Visualizaciones']] #para sacar las correlaciones de Visualizaciones
corr_visual.sort_values(ascending=False, by = 'Visualizaciones')

In [ ]:
features_selecc = ['Visualizaciones necesarias para suscribirse al canal', 'Porcentaje de clics de las impresiones (%)', 'Visualizaciones necesarias para compartir el video'] #seleccionar las 3 mejores

model.fit(X_train[features_selecc], y_train)

In [ ]:
GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=11),
             param_grid={'max_depth': [None, 2, 3, 4]},
             scoring='neg_mean_squared_error')

predict() usando el modelo de las 3 variables

In [ ]:
X_train = X_train.assign(VAR1_VAR2_VAR3=model.predict(X_train[features_selecc]))
X_test = X_test.assign(VAR1_VAR2_VAR3=model.predict(X_test[features_selecc]))

In [ ]:
#X_train.head()

RE-FIT DEL MODELO CON LA NUEVA VARIABLE 

In [ ]:
regressor = XGBRegressor()
model = regressor.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
# Esto está copiado de arriba

test = pd.DataFrame({'Predicted views':y_pred, 'Actual views':y_test})

fig= plt.figure(figsize=(16,8))

test = test.reset_index() # REVISAR ESTE PASO Y EL DE DESPUÉS
test = test.drop(['index'],axis=1)

plt.plot(test[:50]) #revisar si tengo que coger sobre 50 o más bien la mitad de los datos
plt.legend(['Actual views','Predicted views'])

In [ ]:
import pickle
with open('../models/modelo_DT.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

# 4. LINEAR REGRESSION

Hands On Machine Learning: "The simplest and most commonly used."

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('../data/processed.csv')

In [ ]:
#df.info()

In [ ]:
""" plt.figure(figsize=(14,8))
bars = df.corr()['Visualizaciones'].sort_values(ascending=False).plot(kind='bar') """

In [ ]:
X = df.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = df['Visualizaciones']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33, shuffle=True)

Por el momento, los datos consisten en valores pequeños y grandes. Hay algunas columnas que tienen valores medianos de más de 2650. Como estas varían tanto en tamaño, confunden en modelo. Para ponerlos todos en una escala uniforme, tengo que usar la función StandardScaler(). Es importante destacar que usamos fit_transform() en nuestros datos de X_train, pero solo usamos fit() en nuestros datos de X_test. No hacer esto puede causar una "fuga de datos".

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Crear el Modelo de Regresión lineal

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
LinearRegression()

Predicts

In [ ]:
y_pred = model.predict(X_test)
#y_pred

Root Mean Squared Error (pasar a evaluation después)

In [ ]:
print('RMSE: ', np.sqrt(mean_squared_error(y_test, y_pred))) 

MÁS EVALUACION (Pasar tb a evaluation)

In [ ]:
test = pd.DataFrame({'Predicted VIEWS':y_pred, 'Actual VIEWS':y_test})
fig= plt.figure(figsize=(16,8))

test = test.reset_index()
test = test.drop(['index'],axis=1)

plt.plot(test[:40])
plt.legend(['Actual VIEWS','Predicted VIEWS'])

# MALDITOS OUTLIERS

In [ ]:
import pickle
with open('../models/modelo_LinR.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

# 5. XGBREGRESSOR CON GRIDSEARCHCV

Hands On Machine Learning: "It is worth noting that an optimized implementation of Gradient Boosting is available
in the popular python library XGBoost, which stands for Extreme Gradient Boosting."

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

In [ ]:
df = pd.read_csv('../data/processed.csv')

In [ ]:
df = df.loc[df['Visualizaciones']<35000,:] #quito 2 outliers para ver que tal

In [ ]:
X = df.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = df['Visualizaciones']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33, shuffle=True)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
regressor = XGBRegressor()
model = regressor.fit(X_train, y_train)
y_pred = model.predict(X_test)

Visualización

In [ ]:
test = pd.DataFrame({'Predicted views':y_pred, 'Actual views':y_test})
fig= plt.figure(figsize=(16,8))
test = test.reset_index()
test = test.drop(['index'],axis=1)
plt.plot(test[:50])
plt.legend(['Actual views','Predicted views'])

In [ ]:
model.get_params() #para ver que parametros tiene

En la celda de abajo se hizo el GridSearchCV, ahora comentado.

In [ ]:
""" param_grid = dict(
    n_jobs=[-1],
    learning_rate=[0.1, 0.5],
    objective=['reg:squarederror'],
    max_depth=[5, 10, 15], 
    n_estimators=[100, 500, 1000],
    subsample=[0.2, 0.8, 1.0],
    gamma=[0.05, 0.5],
    scale_pos_weight=[0, 1],
    reg_alpha=[0, 0.5],
    reg_lambda=[1, 0],
)

model = XGBRegressor(random_state=1, verbosity=1)

grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='neg_root_mean_squared_error',
                           )

best_model = grid_search.fit(X_train, y_train)
print('Mejores parametros:', best_model.best_params_) """

FIT DE LOS MEJORES PARÁMETROS DEL MODELO

In [ ]:
regressor = XGBRegressor(
    gamma=0.05,
    learning_rate=0.1,
    max_depth=5,
    n_estimators=100,
    objective='reg:squarederror',
    subsample=0.2,
    scale_pos_weight=0,
    reg_alpha=0,
    reg_lambda=1
)
model = regressor.fit(X_train, y_train)
y_pred = model.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_pred)) #EVALUATION

Visualización del modelo mejorado

In [ ]:
test = pd.DataFrame({'Predicted value':y_pred, 'Actual value':y_test})
fig= plt.figure(figsize=(16,8))
test = test.reset_index()
test = test.drop(['index'],axis=1)
plt.plot(test[:100])
plt.legend(['Actual value','Predicted value'])

In [ ]:
import pickle
with open('../models/modelo_XGB.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

YAML por que es el mejor modelo:

In [ ]:
import yaml
with open('../models/modelo_config_def.yaml', 'w') as c:
    yaml.dump(model, c)

# EXTRA. TEST PARA SACAR EL MEJOR MODELO E INTENTAR MEJORARLO

In [ ]:
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lars
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.svm import LinearSVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.isotonic import IsotonicRegression
from sklearn.ensemble import RandomForestRegressor

CARGAMOS EL DATA

In [ ]:
df = pd.read_csv('../data/processed.csv')

In [ ]:
#df['Visualizaciones']

QUITANDO OUTLIERS (para ayudar al modelo)

In [ ]:
df = df.loc[df['Visualizaciones']<35000,:] #quito 2 outliers para ver que tal

Train y test

In [ ]:
X = df.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = df['Visualizaciones']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33, shuffle=True)

Escalando los datos

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

SELECCIÓN DE MODELO

In [ ]:
regressors = {
    "XGBRegressor": XGBRegressor(),
    "RandomForestRegressor": RandomForestRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(),
    "GaussianProcessRegressor": GaussianProcessRegressor(),
    "SVR": SVR(),
    "NuSVR": NuSVR(),
    "LinearSVR": LinearSVR(),
    "KernelRidge": KernelRidge(),
    "LinearRegression": LinearRegression(),
    "Ridge":Ridge(),
    "Lars": Lars(),
    "TheilSenRegressor": TheilSenRegressor(),
    "HuberRegressor": HuberRegressor(),
    "PassiveAggressiveRegressor": PassiveAggressiveRegressor(),
    "ARDRegression": ARDRegression(),
    "BayesianRidge": BayesianRidge(),
    "ElasticNet": ElasticNet(),
    "OrthogonalMatchingPursuit": OrthogonalMatchingPursuit(),
}

Esto de abajo recorre cada uno de los modelos, lo ajusta usando los datos de X_train y y_train, y luego genera predicciones de X_test y calcula el RMSE medio de 10 rondas de Cross Validation. Eso nos dará el RMSE para los datos de X_test, más el RMSE promedio para el conjunto de datos de entrenamiento.

In [ ]:
df_models = pd.DataFrame(columns=['model', 'run_time', 'rmse', 'rmse_cv'])

for key in regressors:

    print('*',key)

    start_time = time.time()

    regressor = regressors[key]
    model = regressor.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    scores = cross_val_score(model, 
                             X_train, 
                             y_train,
                             scoring="neg_mean_squared_error", 
                             cv=10)

    row = {'model': key,
           'run_time': format(round((time.time() - start_time)/60,2)),
           'rmse': round(np.sqrt(mean_squared_error(y_test, y_pred))),
           'rmse_cv': round(np.mean(np.sqrt(-scores)))
    }

    df_models = df_models.append(row, ignore_index=True)

Abajo se ve que al ordenar los resultados del dataframe df_models, se puede ver el rendimiento de cada modelo. La puntuación RMSE de x obtenida de los modelo/s anterior/es se mejoró aquí y ahora se reduce a x. La noticia más importante es que la selección de modelos ha identificado x modelos mucho más efectivos. 

Al quitar los outliers, el modelo PassiveAggressiveRegressor() logró el mejor RMSE medio de 629 y el modelo HuberRegressor un RMSE de 624.

In [ ]:
df_models.head(20).sort_values(by='rmse_cv', ascending=True)

Evaluar el modelo de mejor rendimiento (tengo que pasarlo al evaluation.py correspondiente)

In [ ]:
regressor = PassiveAggressiveRegressor()
model = regressor.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
test = pd.DataFrame({'Predicted views':y_pred, 'Actual views':y_test})
fig= plt.figure(figsize=(16,8))
test = test.reset_index()
test = test.drop(['index'],axis=1)
plt.plot(test[:50])
plt.legend(['Actual views','Predicted views'])

In [ ]:
import pickle
with open('../models/modelo_RFR.pkl', 'wb') as archivo:
    pickle.dump(model, archivo)

Mejorar los hiperparámetros del modelo:

Para ajustar el modelo PassiveAggressiveRegressor() (o cualquier modelo), hay que mirar qué hiperparámetros están disponibles para ajustar con el model.get_params().

In [ ]:
model.get_params()

Hay que seleccionar algunos de los hiperparámetros y agregarlos a un dict() y asignándolos a param_grid. Luego se ejecuta la función GridSearchCV para probar cada combinación única de hiperparámetros y así encontrar los hiperparámetros óptimos del modelo. (Ahora está comentado porque ya se pensó el mejor modelo con esos parámetros)

In [ ]:
""" param_grid = dict(
 C= [1.0, 1.1, 1.2, 1.3],
 average= [False],
 early_stopping= [False],
 epsilon= [0.1, 0.2, 0.3],
 fit_intercept= [True],
 loss= ['epsilon_insensitive', 'squared_epsilon_insensitive'],
 max_iter= [600, 800, 1000, 1200],
 n_iter_no_change= [3,5,7],
 random_state= [None],
 shuffle= [True],
 tol= [0.001, 0.002, 0.0004],
 validation_fraction= [0.1],
 verbose= [0],
 warm_start= [False])

model = PassiveAggressiveRegressor(random_state=33)

grid_search = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='neg_root_mean_squared_error',
                           )

best_model = grid_search.fit(X_train, y_train)
print('Best parameters:', best_model.best_params_) """

FIT del mejor modelo elegido

In [ ]:
regressor = PassiveAggressiveRegressor(C=1.1,
 average= False,
 early_stopping= False,
 epsilon= 0.1,
 fit_intercept= True,
 loss= 'epsilon_insensitive',
 max_iter= 600,
 n_iter_no_change= 7,
 random_state= None,
 shuffle= True,
 tol= 0.001,
 validation_fraction= 0.1,
 verbose= 0,
 warm_start= False)

model = regressor.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('RMSE:',round(np.sqrt(mean_squared_error(y_test, y_pred))))
print('RMSE_CV:',round(np.mean(np.sqrt(-scores)))) 

DESPUÉS DE 40 MINUTOS DE PENSAR, NO CONSEGUIMOS MEJORAR. EL TEST YA HABÍA DADO LA MEJOR OPCIÓN.

# 6. [NO SUPERVISADO]

# [SUB] SUPPORT VECTOR MACHINE (Support Vector Regressor) [no parece que vaya a funcionar]

In [ ]:
""" import numpy as np
import matplotlib.pyplot as plt """

In [ ]:
""" X = df.drop(columns=['Visualizaciones', 'Título del vídeo'])
y = df['Visualizaciones'] """

""" X = np.sort(5*np.random.rand(200,1),axis=0)
Y = np.sin(X).ravel()
Y[::5] += 3*(0.5 - np.random.rand(40)) """

In [ ]:
""" plt.scatter(X,y, color="darkorange", label="data") """
# ValueError: x and y must be the same size

In [ ]:
""" 
from sklearn.svm import SVR

C=1e3

svr_lin = SVR(kernel="linear", C=C)
svr_rbf = SVR(kernel="rbf", C=C, gamma=0.1)
svr_pol = SVR(kernel="poly", C=C, degree=3) """

In [ ]:
""" y_lin = svr_lin.fit(X,y).predict(X)
y_rbf = svr_rbf.fit(X,y).predict(X)
y_pol = svr_pol.fit(X,y).predict(X) """

In [ ]:
""" lw = 2

plt.figure(figsize=(16,9))
plt.scatter(X,y,color="darkorange", label ="data")

plt.plot(X,y_lin, color="navy", lw = lw, label = "SVM Lineal")
plt.plot(X,y_rbf, color="c", lw=lw, label="SVM Radial")
plt.plot(X,y_pol, color="cornflowerblue", label="SVM Polinómico")

plt.xlabel("x")
plt.ylabel("y")

plt.title("Support Vector Regression")
plt.legend()
plt.show() """